## MNIST Datasets

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
seed = 2022
np.random.seed(seed)
tf.random.set_seed(seed)

TypeError: bases must be types

In [4]:
import json

with open('../kaggle.json') as f:
    kaggle = json.load(f)
username = kaggle['username']
kaggle_key = kaggle['key']

In [6]:
kaggle_key

'8b4eff52fc73ee047b5a83e71962d43a'

In [7]:
!wget https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filltered.zip

'wget'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [8]:
!unzip cats_and_dogs_filtered.zip > /dev/null

������ ��θ� ã�� �� �����ϴ�.


In [11]:
!kaggle datasets download -d birajsth/cats-and-dogs-filtered

'kaggle'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.
